<a href="https://colab.research.google.com/github/workprofile1/ProgrammingAI/blob/main/Sentiment_Analysis_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description:


*   This notebook is programmed for sentiment Analysis (Ref: Fine-tuning BERT Model for Sentiment Analysis)

*   It uses Transformers for fine tuning with version DistilBert from hugging Face (kaggle Transformers didnot work)
*   It uses combination of two datasets:
      *   Amazon Reviews
      *   Imdb Movie Reviews

*   Training Loss lowers as trained with more no. of epochs (So approves paper)
*   validation loss is almost constant up till 5 epochs (Opposes paper)

In [4]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = "/content/drive/MyDrive/amazon reviews/test.csv"
df2 = pd.read_csv(file_path,header=None,names=['sentiment','title','review'])
df2=df2.iloc[:10000]
df2

In [25]:
# prompt: drop title column

df2 = df2.drop('title', axis=1)
df2

,sentiment,review
0,2,My lovely Pat has one of the GREAT voices of h...
1,2,Despite the fact that I have only played a sma...
2,1,I bought this charger in Jul 2003 and it worke...
3,2,Check out Maha Energy's website. Their Powerex...
4,2,Reviewed quite a bit of the combo players and ...
...,...,...
9995,2,"This was a wonderful read on poetry, and I esp..."
9996,1,"The device works well enough, but I am disappo..."
9997,2,"My daughter loves this, she is six months old ..."
9998,1,I purchased this item for my 6 month old baby ...


In [6]:
temp_data = pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv")
df = temp_data.iloc[:10000]
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
9995,"Fun, entertaining movie about WWII German spy ...",positive
9996,Give me a break. How can anyone say that this ...,negative
9997,This movie is a bad movie. But after watching ...,negative
9998,This is a movie that was probably made to ente...,negative


In [26]:
df2['sentiment'] = encoder.fit_transform(df['sentiment'])

In [27]:
df['sentiment']= encoder.fit_transform(df['sentiment'])

<ipython-input-27-3326e32b333b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment']= encoder.fit_transform(df['sentiment'])


In [28]:

combined_df = pd.concat([df, df2], ignore_index=True)
combined_df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
19995,"This was a wonderful read on poetry, and I esp...",1
19996,"The device works well enough, but I am disappo...",0
19997,"My daughter loves this, she is six months old ...",0
19998,I purchased this item for my 6 month old baby ...,0


In [29]:
df=combined_df.copy()

In [30]:
# prompt: shuffle data

from sklearn.utils import shuffle

df = shuffle(df)

In [31]:
encoder = LabelEncoder()
Y = encoder.fit_transform(df['sentiment']).tolist()

print(type(Y))

<class 'list'>


In [32]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


In [33]:
# Split dataset into train, validation, and test sets
train_texts, temp_texts, train_labels, temp_labels = train_test_split(df['review'], Y, test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

print(f"Size of Y: {len(Y)}")

# Get the sizes of each split
train_texts_size = len(train_texts)
temp_texts_size = len(temp_texts)
train_labels_size = len(train_labels)
temp_labels_size = len(temp_labels)

# Print the sizes
print(f"Size of train_texts: {train_texts_size}")
print(f"Size of temp_texts: {temp_texts_size}")
print(f"Size of train_labels: {train_labels_size}")
print(f"Size of temp_labels: {temp_labels_size}")

# Get the sizes of each split
val_texts_size = len(val_texts)
test_texts_size = len(test_texts)
val_labels_size = len(val_labels)
test_labels_size = len(test_labels)

# Print the sizes
print(f"Size of val_texts: {val_texts_size}")
print(f"Size of teest_texts: {test_texts_size}")
print(f"Size of val_labels: {val_labels_size}")
print(f"Size of test_labels: {test_labels_size}")

Size of Y: 20000
Size of train_texts: 14000
Size of temp_texts: 6000
Size of train_labels: 14000
Size of temp_labels: 6000
Size of val_texts: 3000
Size of teest_texts: 3000
Size of val_labels: 3000
Size of test_labels: 3000


In [34]:
# Tokenize the Dataset
def tokenize_dataset(texts, labels, tokenizer, max_length=128):
    inputs = tokenizer(texts, max_length=max_length, padding="max_length", truncation=True, return_tensors="tf")
    #print(inputs)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    print(input_ids)
    print(attention_mask)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)  # Explicitly set dtype
    print(labels)
    return input_ids, attention_mask, labels

In [35]:
# Tokenize datasets
train_input_ids, train_attention_mask, train_labels = tokenize_dataset(train_texts.to_list(), train_labels, tokenizer)
val_input_ids, val_attention_mask, val_labels = tokenize_dataset(val_texts.to_list(), val_labels, tokenizer)
test_input_ids, test_attention_mask, test_labels = tokenize_dataset(test_texts.to_list(), test_labels, tokenizer)

tf.Tensor(
[[  101  1000  1996 ...  7570 19473   102]
 [  101  2023  2003 ...  2843  1997   102]
 [  101  2023  3185 ...  2001  2130   102]
 ...
 [  101  2023  2003 ...  1996  4898   102]
 [  101  1045  2123 ...  2054  2009   102]
 [  101  1000  6031 ...     0     0     0]], shape=(14000, 128), dtype=int32)
tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 0 0 0]], shape=(14000, 128), dtype=int32)
tf.Tensor([0 0 0 ... 1 0 1], shape=(14000,), dtype=int32)
tf.Tensor(
[[  101  9202  1010 ...     0     0     0]
 [  101  2023  4031 ...     0     0     0]
 [  101  1045  2572 ...     0     0     0]
 ...
 [  101  2023  2003 ...     0     0     0]
 [  101  1996  4031 ...     0     0     0]
 [  101  2053  2982 ... 28821  2982   102]], shape=(3000, 128), dtype=int32)
tf.Tensor(
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 1 1 1]], shape=(3000, 128), dtype=int

In [36]:
train_input_ids_np = train_input_ids.numpy()
train_attention_mask_np = train_attention_mask.numpy()
train_labels_np = train_labels.numpy()
val_input_ids_np = val_input_ids.numpy()
val_attention_masks_np = val_attention_mask.numpy()
val_labels_np = val_labels.numpy()
test_input_ids_np = test_input_ids.numpy()
test_attention_mask_np = test_attention_mask.numpy()
test_labels_np = test_labels.numpy()

# Save the NumPy array to a file
np.save('train_input_ids.npy', train_input_ids_np)
np.save('train_attention_mask.npy', train_attention_mask_np)
np.save('train_labels.npy', train_labels_np)
np.save('val_input_ids.npy', val_input_ids_np)
np.save('val_attention_mask.npy', val_attention_masks_np)
np.save('val_labels.npy', val_labels_np)
np.save('test_input_ids.npy', test_input_ids_np)
np.save('test_attention_mask.npy', test_attention_mask_np)
np.save('test_labels.npy', test_labels_np)


# using gpu for training

In [37]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [38]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [39]:

train_input_ids = tf.convert_to_tensor(np.load('train_input_ids.npy'))
train_attention_mask = tf.convert_to_tensor(np.load('train_attention_mask.npy'))
train_labels = tf.convert_to_tensor(np.load('train_labels.npy'))

val_input_ids = tf.convert_to_tensor(np.load('val_input_ids.npy'))
val_attention_mask = tf.convert_to_tensor(np.load('val_attention_mask.npy'))
val_labels = tf.convert_to_tensor(np.load('val_labels.npy'))

test_input_ids = tf.convert_to_tensor(np.load('test_input_ids.npy'))
test_attention_mask = tf.convert_to_tensor(np.load('test_attention_mask.npy'))
test_labels = tf.convert_to_tensor(np.load('test_labels.npy'))

In [40]:
def create_data_loader(input_ids, attention_mask, labels, batch_size=16):
    dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": input_ids, "attention_mask": attention_mask}, labels))
    dataset = dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [41]:
def train_model(model, train_data, val_data, epochs=5, learning_rate=5e-5):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.BinaryCrossentropy()  # Binary crossentropy for binary classification
    metric = tf.keras.metrics.BinaryAccuracy()  # Binary accuracy metric

    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    history = model.fit(train_data, validation_data=val_data, epochs=epochs)
    return history

In [42]:
train_data = create_data_loader(train_input_ids, train_attention_mask, train_labels)
val_data = create_data_loader(val_input_ids, val_attention_mask, val_labels)
test_data = create_data_loader(test_input_ids, test_attention_mask, test_labels)

In [43]:
# Train the model
train_model(model, train_data, val_data)

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch 1/5
875/875 [==============================] - 214s 226ms/step - loss: 0.6544 - binary_accuracy: 0.6139 - val_loss: 0.6993 - val_binary_accuracy: 0.6627
Epoch 2/5
875/875 [==============================] - 201s 229ms/step - loss: 0.6589 - binary_accuracy: 0.5982 - val_loss: 0.6213 - val_binary_accuracy: 0.6557
Epoch 3/5
875/875 [==============================] - 201s 230ms/step - loss: 0.6236 - binary_accuracy: 0.6473 - val_loss: 1.2354 - val_binary_accuracy: 0.6613
Epoch 4/5
875/875 [==============================] - 194s 221ms/step - loss: 0.5727 - binary_accuracy: 0.7172 - val_loss: 0.7173 - val_binary_accuracy: 0.6610
Epoch 5/5
188/188 [==============================] - 14s 72ms/step - loss: 0.6921 - binary_accuracy: 0.5800
Test Accuracy: 0.5800
